In [112]:
# Extracts tender zip files from links

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def click_button(driver, wait, button_text):
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, f"//input[@value='{button_text}']")))
        button.click()
    except Exception as e:
        print(f"Error while clicking button '{button_text}': {e}")

def get_tenders(link):
    driver = webdriver.Chrome()

    # tender webpage
    driver.get(link)

    # wait for button to be clickable
    wait = WebDriverWait(driver, 10)

    # password protected, click button for information only version
    click_button(driver, wait, "Download for Information Only")

    # download zip
    click_button(driver, wait, "Download Documents")

    # enough time to download files before quitting
    time.sleep(5)
    driver.quit()

get_tenders("https://www.tenders.wa.gov.au/watenders/tender/request/request-spec-docs.do?CSRFNONCE=687D24C86AA5CD5A167CC3C60BDDD821&tenderId=57321&method=downloadViaHTML")